##### <b>Multiple beam generation</b>

This script generates multiple beams from individual input channels in a single TPM. 

Signals are defined by their ADC input port, numbered from 0 to 31. 
<ul>
    <li> Only one signal from each ADC pairs (one antenna) can be used</li>
    <li> If two signals from one antenna are both specified, their sum is used for both.</li>
    <li> the used signal is copied to both polarizations</li>
    <li> Each signal can be delayed using the <it>staticTimeDelays</it> attribute</li>
</ul>

Up to 8 beams can be defined with the current firmware. All beams have the same bandwidth and sky frequency. 

In the example below 4 beams are defined, for Y polarization in antennas 5-8. They are delayed by [0,1,2,3] samples. Each beam starts at 229.30 MHz and ends at 235.55 MHz (channels 294-301).

 Local parameters
 <ul>
    <li> csp_ingest_ip: IP address of the CSP ingest port</li>
    <li> lmc_ip: IP address of the LMC DAQ system</li>
    <li> input ADCs: ADC input channel (0-31, 0-1 for antenna 1, 30-31 for antenna 16)</li>
    <li> delays: in nanoseconds, rounded to ADC samples (1.25 ns), for each signal. One for each ADC input.
    <li> csp_rounding: Depending on actual signal level, adjust signal level at channelizer output. Nominal value (4) is appropriate for a sinewave with RMS input amplitude, as measured by adcPower, in the range 5.5 to 11. Value is ceil(log2(adcPower/2.8))</li>
    <li> input_frequency: frequency of the input signal. Used to compute the beamformed channel. The beamformer beamforms N channels starting at the first even channel equal or lower to this one </li>
    <li> nof_channels: number of beamformed channels per beam. Must be a multiple of 8
    </ul>


In [1]:
csp_ingest_ip = "10.0.0.99"
lmc_ip = "10.0.0.99"
input_adcs = [9, 11, 13, 15]
delays = [0, 1.25, 2.5, 3.75]
input_frequency = 230e6  # use actual tone frequency
nof_channels = 8
csp_rounding = 4    # adequate for -2:+5 dBm, adcLevel=11:22
start_channel = int(round(input_frequency/800e6*1024))
if start_channel % 2 == 0:
    print(f"Signal is on beamformed channel 0, corresponding to TPM channel {start_channel}")
else: 
    print(f"Signal is on beamformed channel 1, corresponding to TPM channel {start_channel}")
    start_channel = start_channel -1

Signal is on beamformed channel 0, corresponding to TPM channel 294


<b>Definitions</b>

Define the used TANGO devices and constants.
Change list of device proxies and "devices" and "tiles" to actually present and used tiles

Put all devices Online. 

In [2]:
import tango
import time
import json
import numpy as np

from ska_control_model import (
    AdminMode,
    CommunicationStatus,
    HealthState,
    PowerState,
    ResultCode,
    SimulationMode,
    TestMode,
)
# for time conversion
from datetime import datetime,timezone
RFC_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"

# define devices
station = tango.DeviceProxy('low-mccs/station/001')
subrack = tango.DeviceProxy('low-mccs/subrack/0001')
t1 = tango.DeviceProxy('low-mccs/tile/0001')
#t2 = tango.DeviceProxy('low-mccs/tile/0002')
#t3 = tango.DeviceProxy('low-mccs/tile/0003')
#t4 = tango.DeviceProxy('low-mccs/tile/0004')
#t5 = tango.DeviceProxy('low-mccs/tile/0005')
#t6 = tango.DeviceProxy('low-mccs/tile/0006')
#t7 = tango.DeviceProxy('low-mccs/tile/0007')
#t8 = tango.DeviceProxy('low-mccs/tile/0008')

#station.logginglevel=5
devices = [station, subrack, t1,] # t2, t3, t4, t5, t6, t7, t8]
tiles = [t1, ] # t4]

# Put everything online
for d in devices:
    d.adminmode = AdminMode.ONLINE
time.sleep(0.2)
# 
# Tiles must be in Maintenance mode to allow test signal generator
for t in tiles:
    t.adminMode = AdminMode.MAINTENANCE

<b>Turn TPM on</b>
<ul>
    <li>Turn TPM on if not already on.</li>
    <li>Wait for intialization if not already initialised</li> 
    <li>If initialisation succeeds <ul>
        <li>perform initial setup</li>
        <li>start the acquisition.</li></ul>
    <li>Set destination IP addresses</li>
    <li>At the end, print signal level on input ADCs</li></ul>

In [14]:
# CHeck that TPM os on, initialised and synchronised. If not, initialises it
if t1.tileprogrammingstate != 'Synchronised':
    t1.on()
    time.sleep(2)
    t1.initialise()
    
t = 0
while not t1.tileprogrammingstate in ['Initialised', 'Synchronised']:
    print(f"{t}: {t1.tileprogrammingstate}")
    time.sleep(2)
    t = t + 2
    if t > 60:
        break
if t > 60:
    print("Initialisation failed")
elif t1.tileprogrammingstate == 'Initialised':
    print(f"{t}: {t1.tileprogrammingstate}")
    t1.ConfigureStationBeamformer(json.dumps({
        "start_channel": 192,
        "n_channels": 8,
        "is_first": True,
        "is_last": True,
    }))
    start_time = datetime.strftime(datetime.fromtimestamp(int(time.time())+2), RFC_FORMAT)
    t1.StartAcquisition(json.dumps({"start_time": start_time}))
    time.sleep(3)
if t1.tileprogrammingstate == "Synchronised":
    t1.statictimedelays=np.zeros([32],dtype=int)
    t1.channeliserRounding=[csp_rounding]*512
    t1.cspRounding=[0]*384
    t1.SetLmcDownload(json.dumps({"destination_ip": lmc_ip, "mode": "10g"}))
    t1.SetLmcIntegratedDownload(json.dumps({"destination_ip": lmc_ip, "mode": "10g"}))
    t1.Configure40GCore(json.dumps({"core_id": 0, "arp_table_entry": 0, "destination_ip": csp_ingest_ip}))
    t1.Configure40GCore(json.dumps({"core_id": 1, "arp_table_entry": 0, "destination_ip": csp_ingest_ip}))
print(f"{t1.fpgaframetime}: Tile is in state {t1.tileprogrammingstate}")
time.sleep(0.5) # to allow for the total power detector to compute the total power
print(f"Input levels: {t1.adcPower}")

0: Initialised
2023-05-29T15:00:21.356411Z: Tile is in state Synchronised
Input levels: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


Program te test generator to produce null samples except for selected input.
Input ADC can be identified from adcPower attribute above.
Then start the generator.

If adc_inputs or delays are changed, this cell can be re-run

In [15]:
t1.StopBeamformer()
time.sleep(0.1)

static_delays = t1.staticTimeDelays # modify the delays as specified

beamformer_table = []  # Definition of beamformer regions. Flat array
                       # hw_channel, nof_chans, hw_beam, subarray, log_channel, subarray_beam, substation, aperture
beam_channels = []     # logical channels in each region. 2d array
beam = 0
logical_channel = 0
for adc in input_adcs:
    beamformer_table += [start_channel,nof_channels,beam,1,0,1,beam+1,101+beam]
    beam_channels += [[logical_channel, (logical_channel+nof_channels)]]
    logical_channel += nof_channels
    static_delays[adc] = delays[beam]
    beam += 1

# Program beamformer and delays0
t1.SetBeamformerRegions(beamformer_table)
t1.staticTimeDelays = static_delays

# Program beam calibration matrix to send a single antenna signal to each beam

cal_gain = 4.0  # we are using just one antenna, raise the gain
cal_matrix_even = [cal_gain, 0.0, cal_gain, 0.0, 0.0, 0.0, 0.0, 0.0]  # send Xpol to both
cal_matrix_odd  = [0.0, 0.0, 0.0, 0.0, cal_gain, 0.0, cal_gain, 0.0]  # send Ypol to both

# for each antenna the calibration coefficients are zero except for corresponding beam
for antenna in range(16):
    
    cal_coefs = [antenna*1.0] + [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] * 384
    beam = 0
    for adc in input_adcs:
        if adc // 2 == antenna:
            if adc & 1 == 0:
                antenna_coefs = cal_matrix_even
            else:
                antenna_coefs = cal_matrix_odd
            
            for logical_channel in range(beam_channels[beam][0], beam_channels[beam][1]):
                cal_element = logical_channel*8+1
                cal_coefs[cal_element:cal_element+8] = antenna_coefs
        t1.LoadCalibrationCoefficients(cal_coefs)
        beam += 1
        
t1.ApplyCalibration("")

print(t1.fpgaframetime)
start_time = datetime.strftime(datetime.fromtimestamp(time.time()+2), RFC_FORMAT)
t1.StartBeamformer(json.dumps({"start_time": start_time}))
time.sleep(2)
print(f"Beamformer running: {t1.isBeamformerRunning}")
current_rounding = t1.channeliserRounding[0] & 0x7
level = t1.adcPower[input_adcs[0]]
chan_level = level * 1.45 * 2**current_rounding
if chan_level > 110 or chan_level < 50:
    rounding = int(np.ceil(np.log2(level/1.4)))
    if rounding <0:
        rounding = 0
    if rounding > 7: 
        rounding = 7
    t1.channeliserRounding = [rounding]*512
    print(f"Adjust channeliser rounding to {rounding}")
else:
    print(f"Keeping old channeliser rounding {current_rounding}")
print(f"Input level: {t1.adcPower[input_adcs[0]]}")

2023-05-29T15:00:28.547379Z
Beamformer running: True
Adjust channeliser rounding to 3
Input level: 5.762208750591321


In [6]:
t1.senddatasamples(json.dumps({"data_type": "beam"}))

[array([0], dtype=int32), ['SendDataSamples command completed OK']]

In [16]:
t1.stopbeamformer()

[array([0], dtype=int32), ['StopBeamformer command completed OK']]